<a href="https://colab.research.google.com/github/ShounakDas101/multibody/blob/main/IEEE_Cognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import random as rnd

In [ ]:
I=1.0
a=5.0
b=0.5
c=0.6
f=0.3  # it should be a fraction
w2=a/I # it is actually square of angular frequency



N=8 # No of anaisotropy in the potential well = 1+N/2
k=1.38 # boltzmann constant, actual value is 1.380649E-23 Joules/Kelin
T=20    # actual value 300K (at room temp)
KE=0.5*I*w2
U = 15
Amp = 1.3

In [ ]:
def coupled_osc(x, t):
    if   x[0] > 0.5*np.pi:
        x[0]=0.5*np.pi - x[0] # oscillate between -pi/2 to +pi/2
    elif x[0]< -0.5*np.pi:
        x[0]=x[0] - 0.5*np.pi # oscillate between -pi/2 to +pi/2

    return  [x[1], -a*np.sin(x[0]) -b*x[1]  - f*a*np.sin(N*x[0])]

In [ ]:

def flip_osc(x, t):
    if   x[0] > 0.5*np.pi:
        x[0]=0.5*np.pi - x[0] # oscillate between -pi/2 to +pi/2
    elif x[0]< -0.5*np.pi:
        x[0]=x[0] - 0.5*np.pi # oscillate between -pi/2 to +pi/2
    return  [x[1], -a*np.sin(x[0]) -b*x[1] - f*a*np.sin(N*x[0]) + c*np.sin(x[0])]


In [ ]:
# Mind Potential Equation biased because of  various brands in the market
omega_square=a/I
t = np.arange(-np.pi/2,np.pi/2,0.05)   # start,stop,step
t_max=np.pi/2
# New Potential
H0 = U + a*(1-np.cos(t))
H1 = U + a*(1-np.cos(t)) + f*a*(1-np.cos(N*t))
H2 = U + a*(1-np.cos(t)) + f*a*(1-np.cos(N*t)) - 0.5*b*omega_square
H3 = U + a*(1-np.cos(t)) + f*a*(1-np.cos(N*t)) - 0.5*b*omega_square + c*np.cos(t)

In [ ]:

# A plot of the fluid height (position) inside the straw as a function of time.
#plt.plot(t,y)
# plt.show()
plt.title("Potential Well at different situation")
plt.margins(x=0, y=0)
plt.ticklabel_format(style='plain', axis='x')
plt.plot(t*180/np.pi, H0, t*180/np.pi, H1, t*180/np.pi, H2, t*180/np.pi, H3, 'black', linewidth = 1)
plt.xlabel('State of Mind (theta)')
plt.ylabel('Free Energy well (H)')
plt.legend(['Normal Mind', 'Biased Mind', 'Resistive Biased Mind', 'External iInfluenced Mind'])
plt.grid(True)

In [ ]:
# Next, we create an array of the desired times for the solution.
no_of_samples = 12000
tou_0=100
delay =5
# Create an empty NumPy array of zeros
time_array = np.zeros(no_of_samples)
posi_array = np.zeros(no_of_samples)  # displacement or position
velo_array = np.zeros(no_of_samples)  # velocity
enrg_array = np.zeros(no_of_samples)  # energy
mean_array = np.zeros(no_of_samples)  # mean enegy
tou_array  = np.ones(no_of_samples)  # tou
my_time = [0,1e-2] # odeint takes time in terms of array only, h=t1-t0
theta = 0.5*np.pi/2
omega =0
init_value =[theta, omega] # odeint takes init_value in terms of array only
for i in range(0,no_of_samples,1):

    #tou_array[i] = tou_0*np.exp(enrg_array[i]/300)
    # Calculate the average of the last 150 elements
    if i<1500:
        mean_array[i] = enrg_array[i]
    else :
        mean_array[i] = np.mean(enrg_array[i-1500:i])
    # check tou
    if  15.0 tou_0 > 0     :  # here condition should be mean_array[i]/kT <1
            #tou_array[i] = tou_0*np.exp(mean_array[i]/300)
            #c = 0.321       #2*rnd.random()

            # Now calculate the free energy Hi
            enrg_array[i]= U + 0.5*I*omega*omega + a*(1-np.cos(theta))+ f*a*(1-np.cos(N*theta))- 0.5*b*omega*omega + c*np.cos(theta)

            #omega1=omega + c*np.sin(theta)*1e-2/I-b
            #theta1=theta + omega*1e-2
            #omega1 =  np.sqrt(2*c*np.cos(theta)/(I-b) + omega*omega) # energy injected to the system is converted to KE in the system
            #theta1 = theta + omega*1e-2
            #init_value =[theta, omega]
            solution = odeint(flip_osc, init_value, my_time)
            posi_array[i]=solution[1,0]
            velo_array[i]=solution[1,1]
            theta=posi_array[i]
            omega=velo_array[i]
            init_value=[theta, omega]

            tou_0 = tou_0 -1
            #print("flip",tou_array[i])
    else:
            # Now calculate the free energy Hi
            #tou_array[i] = tou_0*np.exp(mean_array[i]/300)
            enrg_array[i]=U + 0.5*I*omega*omega + a*(1-np.cos(theta))+ f*a*(1-np.cos(N*theta))- 0.5*b*omega*omega
            solution = odeint(coupled_osc, init_value, my_time)
            posi_array[i]=solution[1,0]
            velo_array[i]=solution[1,1]
            theta=posi_array[i]
            omega=velo_array[i]
            init_value=[theta, omega]

            tou_0 = delay
            #print("coupled" , tou_array[i])

    time_array[i]=i*1e-2


    #print (last_five_average,theta,omega)


In [ ]:

from matplotlib import colors
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('time (msec)')
ax1.set_ylabel('angular displacement and velocity', color=color)
ax1.plot(time_array, posi_array, time_array, mean_array,  'orange', linewidth = 0.5)
ax1.tick_params(axis='y', labelcolor=color)
ax1.legend(['Position', 'Velocity'])
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Energy', color='black')  # we already handled the x-label with ax1
ax2.plot(time_array, enrg_array, color='black')
ax2.tick_params(axis='y', labelcolor='black')
#ax2.legend(['Energy'])
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()


In [ ]:

# A plot of the fluid height (position) inside the straw as a function of time.
plt.plot(time_array[100:12000], 180*posi_array[100:12000]/np.pi,'red', linewidth = 1)
plt.xlabel('time (s)')
plt.ylabel('posi_array (theta)')
plt.ticklabel_format(style='plain', axis='x', scilimits=(0,0))
plt.grid(True)

In [ ]:
# A plot of the fluid height (position) inside the straw as a function of time.
plt.plot(time_array[2000:12000], mean_array[2000:12000],'red', linewidth = 1)
plt.xlabel('time (s)')
plt.ylabel('Mean energy (J)')
plt.ticklabel_format(style='plain', axis='x', scilimits=(0,0))
plt.grid(True)